# CREATE CALIBRATION

In [ ]:
import os
import numpy as np
import tkinter as tk
from tkinter import filedialog
import image_handler as tif


default_data_path = f'{os.getcwd()}{os.sep}raw_data'
poni = f'{os.getcwd()}{os.sep}calibration{os.sep}'
# filename = f'{default_data_path}{os.sep}TT5mm-01-benzeneTPP_60min_flip.tif'


"""Inputs"""

"""End Inputs"""


%gui qt

### Find image and flip it if needed

In [2]:
"""Select file using GUI prompt (can uncomment filename above and skip)"""
# filename = '/home/etortoric/Documents/GitHub/GIWAXS/raw_data/TT5mm-01-benzeneTPP_60min_flip-dz.tif' 
root = tk.Tk()
root.withdraw()
root.update()
cal_file = filedialog.askopenfilename(initialdir=default_data_path, title='Select Files to Flip', filetypes=(('TIFFs', '*.tif',), ('all', '*.*')))
root.destroy()
print(f'Using: {filename}\n    as calibration image')

"""If in the unflipped folder, we'll need to flip it"""
if f'{os.sep}unflipped{os.sep}' in filename.replace('/', os.sep):
    print('Flipping the image for pyFAI-calib2')
    # takes tif at 'filename' and rotates 180degrees and then saves in the folder 'save_path'
    # returns new filename of the flipped image
    cal_file = tif.flip(filename=filename, save_path=default_data_path)

print(f'Flipped image location is\n    {filename}')
r, c = tif.size(filename)
print(f'The calibration image is {r} x {c} pixels')
# tif.view_tif(cal_file)

Using: /home/etortoric/Documents/GitHub/GIWAXS/raw_data/SiBe_on_GIWAXmount_CenteredInBeam_flip.tif
    as calibration image
Flipped image location is
    /home/etortoric/Documents/GitHub/GIWAXS/raw_data/SiBe_on_GIWAXmount_CenteredInBeam_flip.tif
The calibration image is 1102 x 3070 pixels


### Run calibration GUI

In [ ]:
"""Can skip this if you do not wish to apply dezingering"""

apply_dezinger = 3       #  this many times
cutoff = 1.2             #  relative cutoff for (smoothed-original)/smoothed. cuts off below -cutoff and above cutoff * 2
smoothing = 3            #  the standard deviation used in the gaussian smoothing. Larger the smoother

cal_file = remove_zingers_from_file(cal_file, cutoff, smoothing, apply_dezinger)

# tif.view(cal_file)

In [ ]:
"""Select file using GUI prompt (can uncomment filename above and skip)"""
# filename = '/home/etortoric/Documents/GitHub/GIWAXS/raw_data/TT5mm-01-benzeneTPP_60min_flip-dz.tif' 
root = tk.Tk()
root.withdraw()
root.update()
filename = filedialog.askopenfilename(initialdir=default_data_path, title='Select Files to Flip', filetypes=(('TIFFs', '*.tif',), ('all', '*.*')))
root.destroy()
print(filename)

"""If in the unflipped folder, we'll need to flip it"""
if f'{os.sep}unflipped{os.sep}' in filename:
    print('Flipping the image for pyFAI-calib2')
    # takes tif at 'filename' and rotates 180degrees and then saves in the folder 'save_path'
    # returns new filename of the flipped image
    filename = flip_tif(filename=filename, save_path=default_data_path) 